#  SpaceX EDX SQL Server
## Objectives

Writing and executing SQL queries to solve the questions being asked.


#### The dataset used to be queried can be found below.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">SpaceX DataSet</a>


### Connect to the database

Let us first load the SQL extension and establish a connection with the database

As <code> IBM's DB2 guidelines on this notebook doesnt work </code> I will implement the SQL queries using my normal Microsoft SQL Server instance using my own library <code> 'sqlserver' </code> for other info, please  on my libary visit https://github.com/ADGVLOGS/pythonsqlserverclass 


In [1]:
!pip install sqlserver

     |████████████████████████████████| 280 kB 5.3 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287353 sha256=624f64922deba8b01e15c6e08fd3871ab8af37e47c83e027f8b24a9867988e95
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc


### Initialization

the .sqlserver() object parameters stands for 
(ip,portnumber,databasename,username,password)  

In [4]:
import sqlserver as ss 
db = ss.sqlserver('localhost','1433','SpaceX','admin','admin')

AttributeError: ignored

#### Display the names of the unique launch sites  in the space mission


In [3]:
db.GetRecordsOfColumn('select DISTINCT Launch_Site from tblSpaceX','Launch_Site')

NameError: ignored

#### Display 5 records where launch sites begin with the string 'KSC'


In [ ]:
import pyodbc
import pandas as pd 
import numpy as np
conn = pyodbc.connect('Driver={SQL Server};'
                                  'Server=localhost;'
                                  'Database=SpaceX;'
                                  'User ID=admin;Password=admin;')
cursor = conn.cursor()

cursor.execute("select TOP 5 * from tblSpaceX WHERE Launch_Site LIKE 'KSC%'")
columns = [column[0] for column in cursor.description]
results = []
for row in cursor.fetchall():   
     results.append(dict(zip(columns, row)))

df = pd.DataFrame.from_dict(results)
df

#### Display the total payload mass carried by boosters launched by NASA (CRS)


In [ ]:
TPM = db.GetRecordsOfColumn("select SUM(PAYLOAD_MASS_KG_) TotalPayloadMass from tblSpaceX where Customer = 'NASA (CRS)'",'TotalPayloadMass')
ndf= pd.DataFrame(TPM)
ndf.columns = ['Total Payload Mass']
ndf

#### Display average payload mass carried by booster version F9 v1.1


In [ ]:
APM = db.GetRecordsOfColumn("select AVG(PAYLOAD_MASS_KG_) AveragePayloadMass from tblSpaceX where Booster_Version = 'F9 v1.1'",'AveragePayloadMass')
ndf= pd.DataFrame(APM)
ndf.columns = ['Average Payload Mass']
ndf

#### List the date where the succesful landing outcome in drone ship was acheived.


In [ ]:
SLO = db.GetRecordsOfColumn("select MIN(Date) SLO from tblSpaceX where Landing_Outcome = 'Success (drone ship)'",'SLO')
ndf= pd.DataFrame(SLO)
ndf.columns = ['Date which first Successful landing outcome in drone ship was acheived.']
ndf

#### List the names of the boosters which have success in ground pad  and have payload mass greater than 4000 but less than 6000


In [ ]:
SLO = db.GetRecordsOfColumn("select Booster_Version from tblSpaceX where Landing_Outcome = 'Success (ground pad)' AND Payload_MASS_KG_ > 4000 AND Payload_MASS_KG_ < 6000",'Booster_Version')
ndf= pd.DataFrame(SLO)
ndf.columns = ['Date which first Successful landing outcome in drone ship was acheived.']
ndf

#### List the total number of successful and failure mission outcomes


In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                                  'Server=localhost;'
                                  'Database=SpaceX;'
                                  'User ID=admin;Password=admin;')
cursor = conn.cursor()

cursor.execute("SELECT(SELECT Count(Mission_Outcome) from tblSpaceX where Mission_Outcome LIKE '%Success%') as Successful_Mission_Outcomes,(SELECT Count(Mission_Outcome) from tblSpaceX where Mission_Outcome LIKE '%Failure%') as Failure_Mission_Outcomes")
columns = [column[0] for column in cursor.description]
results = []
for row in cursor.fetchall():   
     results.append(dict(zip(columns, row)))

df = pd.DataFrame.from_dict(results)
df

#### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                                  'Server=localhost;'
                                  'Database=SpaceX;'
                                  'User ID=admin;Password=admin;')
cursor = conn.cursor()

cursor.execute("SELECT DISTINCT Booster_Version, MAX(PAYLOAD_MASS_KG_) AS [Maximum Payload Mass] FROM tblSpaceX GROUP BY Booster_Version ORDER BY [Maximum Payload Mass] DESC")
columns = [column[0] for column in cursor.description]
results = []
for row in cursor.fetchall():   
     results.append(dict(zip(columns, row)))

df = pd.DataFrame.from_dict(results)
df

#### List the records which will display the month names, succesful landing_outcomes in ground pad ,booster versions, launch_site for the months in year 2017


In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                                  'Server=localhost;'
                                  'Database=SpaceX;'
                                  'User ID=admin;Password=admin;')
cursor = conn.cursor()

cursor.execute("SELECT   DateName( month , DateAdd( month , MONTH(CONVERT(date,Date, 105)) , 0 ) - 1 )  as Month, Booster_Version, Launch_Site, Landing_Outcome FROM tblSpaceX WHERE  (Landing_Outcome LIKE N'%Success%') AND YEAR(CONVERT(date,Date, 105)) = '2017'")
columns = [column[0] for column in cursor.description]
results = []
for row in cursor.fetchall():   
     results.append(dict(zip(columns, row)))

df = pd.DataFrame.from_dict(results)
df

#### Rank the  count of  successful landing_outcomes between the date 2010-06-04 and 2017-03-20 in descending order.


In [ ]:
sl = db.GetRecordsOfColumn("SELECT COUNT(Landing_Outcome) AS sl FROM dbo.tblSpaceX WHERE (Landing_Outcome LIKE '%Success%') AND (Date >'04-06-2010') AND (Date < '20-03-2017')",'sl')

ndf= pd.DataFrame(sl)
ndf.columns = ['Successful Landing Outcomes Between 2010-06-04 and 2017-03-20']
ndf

### Author : Ashlin Darius Govindasamy